# Finding Lane Lines on the Road

Below is the code pipeline used to find lanes road images as part of the first project in the Self Driving Car course offered by Udacity. Reflections about the pipeline and its short comings are at the very bottom of the notebook.

In [16]:
# import all packages necessary to make this notebook run
from moviepy.editor import VideoFileClip                                         
from IPython.display import HTML   
import matplotlib.pyplot as plot
import matplotlib.image as mimage
import numpy
import cv2
import os
import math

In [17]:
# define a test image to quickly test image transform functions
test_image = mimage.imread('test_images/solidWhiteRight.jpg')
plot.imshow(test_image)

In [18]:
# paths where test videos are taken and outputed
video_file_path = 'test_videos/'
video_output_path = 'test_videos_output/'

In [19]:
# apply grayscale transform on image
def gray_transform(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image_gray

In [20]:
# apply guassian blur on image.
# applying blur softens the pixels which results in softened edges when applying canny transform
def gaussian_blur(image, kernel_size=5):
    image_blurred = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    return image_blurred

In [21]:
# apply Canny edge detection
# extract edged from an image
def canny_transform(image, min=50, max=200):
    image_canny = cv2.Canny(image, min, max)
    return image_canny

In [22]:
# create a polygon to isolate a region of interest in the image
# areas outside of the polygon will be masked out, which is usefull when we try to identify lanes in the image
# takes the vertices parameter: an array of data points(xy) that corresponds to the sides of the polygon
def region_of_interest(image, vertices):
    mask = numpy.zeros_like(image)   
    
    if len(image.shape) > 2:
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [24]:
# apply houghline transform to image
# the output is the detected lines inside the region of interest mask we define
# the input image is the output image of the region_of_interest function
def houghline_transform(image, rho=0.4, theta=numpy.pi/180, threshold=15, min_line_len=20, max_line_gap=200):
    houghlines = cv2.HoughLinesP(image, rho, theta, threshold, numpy.array([]), min_line_len, max_line_gap)  
    return houghlines

In [25]:
# convenience funtion
# flattens an array of arrays
def flatmap(items):                                                              
    return [y for x in items for y in x]

# returns the slope of a line
def slope(xy):                                                                   
    x1, y1, x2, y2 = xy[0]                                                       
    return (y1 - y2) / (x1 - x2) 

# returns two arrays: [x data points of a line] and [y data points of a line]
def get_points(line):                                                            
    x1, y1, x2, y2 = line[0]                                                     
    return [x1, x2], [y1, y2]  

# returns a tuple of [x data points] and [y data points]
# flattens the result of get_points: [[x1, x2], [x1, x2]] becomes [x1, x2, x1, x2 ...]
def data_points(lines):                                                          
    xs = [x[0] for x in map(get_points, lines)]                          
    ys = [y[1] for y in map(get_points, lines)]                          
    xs = flatmap(xs)                                                             
    ys = flatmap(ys)                                                             
    return xs, ys

# filter houghlines according to their slope
# slope < 0 are allocated in the left data points
# slope > 0 are allocated in the right data points
# returns a tuple with ([left x data points], [left y data points]) , ([right x data points], [right y data points])
def split_lines(houghlines):
    lx, ly= data_points(list(filter(lambda line: -0.899 < slope(line) < -0.5, houghlines)))
    rx, ry= data_points(list(filter(lambda line: 0.899 > slope(line) > 0.5, houghlines)))
    return (lx, ly), (rx, ry)

# draws a line in the image
# receives xy as parameter where xy is a tuple containing [x data points] and [y data points]
# detect the average slope, intercept of the xy data points using polyfit
# use the slope, intercept (m, b) to find the average x1, x2 data points
# y1 is the height of the image and y2 is approx. half of the image
def draw_lines(xy, image, color=(0, 200, 0), thickness=20):
    height, width, _ = image.shape
    x, y = xy
    if len(x) > 0:                                                              
        m, b = numpy.polyfit(x, y, 1)                                          
        y1 = height
        x1 = (y1 - b) / m                                                    
        y2 = height/2 * 1.2                                                     
        x2 = (y2- b) / m
        cv2.line(image, (int(x1), height), (int(x2), int(y2)), color, thickness) 

    return image

In [26]:
# output the result of lane detection pipeline
# apply all the necessary color transforms on image before extracting lines
# draws lines in a copy of original image and merge both as a result 
def process_image(image=test_image):
    image_gray = gray_transform(image)
    image_blur = gaussian_blur(image_gray)
    image_edges = canny_transform(image_blur)
    mask_height, mask_width, _ = image.shape
    image_masked = region_of_interest(image_edges, numpy.array([[(0, mask_height), (mask_width/6, mask_height/2), (mask_width - (mask_width/6), mask_height/2), (mask_width, mask_height)]], dtype=numpy.int32))
    houghlines = houghline_transform(image_masked)
    lines = split_lines(houghlines)
    image_lines = numpy.copy(image)*0                                             
    [draw_lines(xy, image_lines) for xy in lines]
    result = cv2.addWeighted(image, 1, image_lines, 1, 0)             
    return result


In [27]:
plot.imshow(process_image())

In [28]:
# apply process_image pipeline in video images
def process_video(filename):
    output = video_output_path + filename                          
    clip1 = VideoFileClip(video_file_path + filename)                         
    white_clip = clip1.fl_image(process_image)
    white_clip.write_videofile(output, audio=False) 

In [29]:
challenge = 'challenge.mp4'
process_video(challenge)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:08<00:00, 29.41it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 



In [30]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output_path + challenge))

In [31]:
solidWhiteRight = 'solidWhiteRight.mp4'
process_video(solidWhiteRight)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:03<00:00, 62.39it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 



In [33]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output_path + solidWhiteRight))

In [34]:
solidYellowLeft = 'solidYellowLeft.mp4'
process_video(solidYellowLeft)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:11<00:00, 58.41it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 



In [35]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output_path + solidYellowLeft))

### Pipeline description:

The pipeline uses the techniques learned throughout the first lesson in term 1. The two most important ones in my understanding are Canny transform and Houghline transform. These form the base concept that helps in the detection of lanes in the road images.

Canny transform outputs only the strongest edges in the image, which is essential for finding lines with the Houghline transform step in the pipeline. Houghline transform outputs all possible lines found in the canny image. However before and after Canny transform and Houghline transform we need to implement a few steps to output the final result seen in the videos above. In a nutshell these steps are:

- Apply grayscale transform in image to better detect edged in Canny transform. High values (white) represent strong edges, dark values (black) represent soft spots.

- Apply Gaussian blur in order to smooth the pixels. This improves the noise when we apply edge detection with Canny transform.

- Use Canny transform to find strong edges in the image

- Isolate a region of interest in the image where relevant lines should be detected by the Houghline transform. This step masks out unnecessary edges which can be considered lines in the Houghline step.

- Apply Houghline transform to find lines in the region of interest. 

- Split the lines into left side of image and right side of image, where each side corresponds to left and right lane in the road. Line splitting is done by finding its slope value

- Draw lines using numpy.polyfit to find the average slope of x y data points and the x = (y - b) / m to find extrapolate the lines to bottom and center of image. Here y is represented by the y value of our region of interest

### Shortcomings:

The pipeline detects the lanes on the road and displays a line on top of them quite well. However, there are some issues with this implementation:

- The line could have a more smooth feel and flicker less. 
- Areas of the image where lane color blends with background causes the lane to either break or not show at all.
- The code is not adaptive to road conditions. As an example, when the road has inconsistent lane intervals it becomes dificult to detect and draw lines. It's possible to tweak the Houghline parameters to get better results but it doesn't work equaly in all cases.

Another big improvement is the detection of curves along the lines, which is not possible with this implementation.

